<h1>Tarea 1, parte 2</h1>
<h2>Pregunta 1</h2>
Para convertir del sistema de filtros de la HRC al sistema UBVRI, se realiza el ajuste siguiendo la pauta del ejemplo (que transforma del sistema HRC a WFC) de Sirianni et al. (2005):
$$(1): V = F555W + c_{1,1}(V-I)+c_{2,1}(V-I)^2$$
$$(2): I = F814W + c_{2,1}(V-I)+c_{2,2}(V-I)^2$$
Los valores de las constantes se encuentran tabuladas para cada filtro, y dependen del valor $TCOL = F555W - F814W$, el cual también se asumirá como el valor $(V-I)$ para una primera aproximación. Luego se aplica la ecuación (1) y (2) para calcular V e I. Se calcula $TCOL$ con los valores actualizados de $(V-I)$ y se vuelven a aplicar las ecuaciones (1) y (2). Repitiendo el procedimiento, los valores $(V-I)$ cambian cada vez menos. Se comprueba que realizar 6 repeticiones basta para obtener un resultado con mejor precisión que la magnitud de entrada, pero se realizarán un par más en esta ocasión.

Se escribe un catálogo y un CMD con esta nueva información.

In [ ]:
from astropy.io import ascii
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import HessCMD

tbl = ascii.read('n121_match.cat')

def transform(v,i):
	c1f555 = [-0.09,-0.124]
	c2f555 = [0.034,0.018]
	c1f814 = [0.06,0.001]
	c2f814 = [-0.099,0.013]
	for j in range(8):
		tcol = v-i
		v = np.where(tcol<0.6, (tbl['f555wMAG']+c1f555[0]*tcol+c2f555[0]*tcol*tcol), (tbl['f555wMAG']+c1f555[1]*tcol+c2f555[1]*tcol*tcol))
		i = np.where(tcol<0.1, (tbl['f814wMAG']+c1f814[0]*tcol+c2f814[0]*tcol*tcol), (tbl['f814wMAG']+c1f814[1]*tcol+c2f814[1]*tcol*tcol))
	return v,i

tbl['V'], tbl['I'] = transform(tbl['f555wMAG'],tbl['f814wMAG'])

# Genera catalogo
data = [tbl['V'],tbl['I'],tbl['f814wALPHA'],tbl['f814wDELTA']]
ascii.write(data,'n121_ubvri.cat',delimiter='\t')

# Genera plots
HessCMD.plotHess(tbl['V'], tbl['V']-tbl['I'], cbarrtitle='Density', saveas='HessCMD_n121.png', cbarr='Yes')

fig, axis = plt.subplots(figsize=(8,6))
axis.plot(tbl['V']-tbl['I'],tbl['V'],'k.',alpha=0.4,ms=3)
axis.set_xlim(-0.5,2)
axis.set_ylim(16,25)
axis.invert_yaxis()
axis.set_xlabel('$V-I$',fontsize=20)
axis.set_ylabel('$V$',fontsize=20)
plt.savefig('cmd_n121_ubvri', dpi=300)

<img src="cmd_n121_ubvri.png">
<img src="HessCMD_n121.png">

<hr>
<h2>Pregunta 2</h2>
Usando como parámetros de entrada:
<ul>
 <li>Edad = 10.5 Gyr</li>
 <li>Fe/H = -1.56</li>
 <li>Distancia = 61KPc</li>
 <li>$A_V = 0.1272$</li>
</ul>

In [ ]:
from isochrones.dartmouth import Dartmouth_Isochrone
iso = Dartmouth_Isochrone(bands=['V','I'])
model = iso.isochrone(age=10.021189299,feh=-1.56,distance=61000,AV=0.1272)

axis.plot(model.V_mag - model.I_mag,model.V_mag,'g',lw=2)
plt.savefig('iso_n121', dpi=300)

Se obtuvo la isócrona de la imagen (línea verde). Al menos pasa cerca de los puntos obtenidos por fotometría.
<img src="iso_n121.png">
Sin embargo, para una mejro aproximación sería necesaria una magnitud V mayor, de forma que todo el CMD se mueva a la derecha y hacia abajo.

<hr>
<h2>Pregunta 3</h2>
Para calcular el <em>ridge line</em> del set de datos, primero se dividió el eje y en rangos de 0.22 magnitudes, que es lo mínimo posible para que no aparezcan secotres sin puntos. En cada rango se calculó la mediana de V-I y se ensambló un set de coordenadas que contiene (mediana,V). El segmento en el eje y que contiene la rama horizontal (entre 19.4 y 20) fue ignorado para este cálculo.
Posteriormente, se ajustó un polinomio de grado 6.

Se grafica en <strong>azul</strong> la línea ajustada.
Se ajustarán la magnitudes en la última pregunta con tal de poder comparar esta isócrona con el ridge line y un mejor fit.

In [ ]:
s = 0.22
x = []
y1 = np.arange(16.5,19.4,step=s)
y2 = np.arange(20,24.5,step=s)
y = np.append(y1,y2)
for i in y:
	a = np.where((i<tbl['V']) & (tbl['V']<i+s))
	x.append(np.median(tbl['V'][a]-tbl['I'][a]))

p = np.poly1d(np.polyfit(y,x,6))
#axis.plot(x,y,'r',lw=2)
axis.plot(p(y),y,'b',lw=2)
plt.savefig('rline_n121', dpi=300)

<img src="rline_n121.png">

<hr>
<h2>pregunta 4</h2>
Ahora que se tiene la ridge line, se busca una isócrona que se acerque de la mejor manera. Suponiendo que la distancia al cúmulo es el dato mejor conocido, se cambian los valores de edad y metalicidad. Se encuentra que el mejor fit se logra con
<ul>
 <li>Edad: 10.12Gyr</li>
 <li>Fe/H: -1.2</li>
</ul>
<img src="iso_fit_n121.png">
El fit se traza en rojo. Si bien no es el más cercano a la línea azul, adopta una forma cercana. El trabajo de desplazar el gráfico del CMD puede hacerse al cambiar los valores V e I. Haciendo $V = V+0.1$ Se obtiene el siguiente fit:

In [ ]:
plt.close()
fig, axis = plt.subplots(figsize=(8,6))
axis.set_xlim(-0.5,2)
axis.set_ylim(16,25)
axis.invert_yaxis()
axis.set_xlabel('$V-I$',fontsize=20)
axis.set_ylabel('$V$',fontsize=20)

axis.plot(tbl['V']-tbl['I']+0.1,tbl['V']+0.1,'k.',alpha=0.4,ms=3)
axis.plot(p(y)+0.1,y+0.1,'b',lw=2)
axis.plot(model2.V_mag - model2.I_mag,model2.V_mag,'r',lw=2)
plt.savefig('iso_fit2_n121', dpi=300)

<img src="iso_fit2_n121.png">

Otras isócronas que podrían haberse utilizado son las de Teramo y las de Padova. En la literatura, ambas entregan edades mayores de este cúmulo (11.8 Gyr y 11.2 Gyr respectivamente, según <a href="http://iopscience.iop.org/article/10.1088/0004-6256/135/4/1106/pdf">katharina Glatt et al. 2008</a>)